In [1]:
#Playlist scrape and play debug v.84
import csv
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver

In [2]:
#Enter a *playlist* url in between quotes here:
#Please ensure ChromeDriver is installed and it's path accessible
url = 'https://www.youtube.com/playlist?list=PLqwFB_HzJk-bsOCmCUr7j5l9UEBMzgME_'
driver = webdriver.Chrome()
driver.get(url)

In [3]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [4]:
# Bulk code. Gets YouTube video data in four stages.

# Get individual video URLS

data_1 = []
for link in soup.find_all('a', href=True):
    data_1.append(link['href'])

data_1_list = [url for url in data_1 if "/watch?v" in url]
data_list = [url2 for url2 in data_1_list if "&index=" in url2]

addon_result = []
for i in data_list:
    if i not in addon_result:
        addon_result.append(i)

list_1 = []
for i in range(len(addon_result)):
    list_1.append("https://www.youtube.com"+addon_result[i])
    
URL = list_1

# Get individual video Titles
cluttered_words = ["YouTube Home", "Home", "Explore", "Subscriptions", "Library", "History"]

data_2 = []
for name in soup.find_all('a', title=True):
    data_2.append(name['title'])      

list_build = data_2

for item in cluttered_words:
    if item in list_build:
        list_build.remove(item)
        list_2 = list_build
        
Title = list_2

data_3 = []
for label in soup.find_all('a'):
    value = label.get('aria-label')
    if value is not None:
        data_3.append(value)
                
list_3_template = [x for x in data_3 if "ago" in x]

list_3 = []
for x in list_3_template:
     split_results = x.split('ago')
 
     list_3.append(split_results[1])
Duration = list_3


#Get individual video Author

list_4_template = []
for x in data_3:
     split_results = x.split('by')
     list_4_template.append(split_results)
    
list_4 = []               
for x in list_4_template:          
    list_4.append(x)

list_4_template = [x for x in data_3 if "by" in x]

list_4_build = []
for x in list_4_template:
     split_results = x.split('by')
     
     list_4_build.append(split_results[1])

list_4 = []
for author_get in list_4_build:
    words = author_get.split()
    if 'seconds' in words:
        list_4.append(' '.join(words[:-7]))
    else:
        list_4.append(' '.join(words[:-5]))

Author = list_4

In [5]:
# Cleanup list contents and create dataframe (available to export)

x = {'YouTube_Video_URL': URL ,'YouTube_Video_Title': Title , 'Runtime': Duration , 'Author': Author}
df_videodata = pd.DataFrame.from_dict(x, orient='index')
df_videodata = df_videodata.transpose()
df_videos = df_videodata[df_videodata["YouTube_Video_URL"].str.contains("None")== False]

In [6]:
#Display DataFrame
df_videos

,YouTube_Video_URL,YouTube_Video_Title,Runtime,Author
0,https://www.youtube.com/watch?v=e0L3SFnIz1M&li...,Python 3 playlist introduction,"1 minute, 4 seconds",Programming Basics
1,https://www.youtube.com/watch?v=jUKQi9ZSchM&li...,Python keywords and identifiers,"5 minutes, 54 seconds",Programming Basics
2,https://www.youtube.com/watch?v=HwsROiZ6zV8&li...,Python Data Types and Variables,8 minutes,Programming Basics
3,https://www.youtube.com/watch?v=-FLZLZMA6yU&li...,Python 3 Algebraic Operators and Algebraic Exp...,"8 minutes, 16 seconds",Programming Basics
4,https://www.youtube.com/watch?v=X_HdqPsqXQQ&li...,Python data type conversions,"4 minutes, 44 seconds",Programming Basics
5,https://www.youtube.com/watch?v=jyMm7LUvRMs&li...,Python string data type,"14 minutes, 34 seconds",Programming Basics
6,https://www.youtube.com/watch?v=6ai_gbGKo3w&li...,Introduction to Webscraping using Python 3 and...,"12 minutes, 21 seconds",Programming Basics
7,https://www.youtube.com/watch?v=EurAoPdyyL8&li...,Python 3 requests module,"12 minutes, 56 seconds",Programming Basics
8,https://www.youtube.com/watch?v=GWXpWU3d23M&li...,Using BeautifulSoup and Python to navigate an ...,22 minutes,Programming Basics
9,https://www.youtube.com/watch?v=Fin_f2uqmK4&li...,BeautifulSoup find() and find_all() methods,16 minutes,Programming Basics


In [ ]:
# Launchs VLC to run playlist accordingly.
# Check your paths! The directory being written too should be accessible by VLC.

import vlc
video = list_1

with open("C:\\Users\\Admin\pythonwork\\write.m3u","w") as f:
    wr = csv.writer(f,delimiter="\n")
    for ele in video:
        wr.writerow([ele+","])
        
import subprocess

subprocess.Popen(['C:\\Program Files\\VideoLAN\\VLC\\vlc.exe', '--playlist-autostart', 'write.m3u'])